In [1]:
%matplotlib
import os
import glob
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from complexity_preprocess import load_scouts, create_inducedfields
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import plot_model

Using matplotlib backend: Qt5Agg


Using TensorFlow backend.


In [2]:
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

<IPython.core.display.Javascript object>

In [18]:
def get_ml_data(root, subject, sf0, sf1, scouts):

    subject_name = 'Sub' + str(subject)
    data_f = create_inducedfields(root, subject_name, sf1, scouts)
    data_b = create_inducedfields(root, subject_name, sf0, scouts)
    
    n_timesamples = np.size(data_f, 1)
    train_size = int(n_timesamples * 0.9)
    test_size = n_timesamples - train_size
    
    train_f = data_f[0, :train_size, :] # To ensure same ML data for both classes, 'F' and 'B'
    train_b = data_b[0, :train_size, :]
    test_f = data_f[0, train_size:, :]
    test_b = data_b[0, train_size:, :]
    
    x_train = np.concatenate((train_f, train_b), axis=0)
    y_train = np.zeros((train_size * 2, 1))
    y_train[:train_size] = 1
    y_train[train_size:] = 0
    x_train, y_train = shuffle(x_train, y_train)
#     x_train, y_train = shuffle(x_train, y_train, random_state=0)
    
    x_test = np.concatenate((test_f, test_b), axis=0)
    y_test = np.zeros((test_size * 2, 1))
    y_test[:test_size] = 1
    y_test[test_size:] = 0
    
    return x_train, y_train, x_test, y_test

def create_model():
    model = Sequential()
    model.add(Dense(10, input_dim=2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [19]:
%%notify
# Parameters
root = 'D:/MATLAB/Perception/Trials/'
subject, sf1, sf0, nreps = 5, 'F', 'B', 5
fscouts = 'D:/Research/Academic/PhD/Perception/Results/SourceLocal/scout_V1_V2.mat'

# Load scout vertices
scouts = load_scouts(fscouts)

# Load ML data
x_train, y_train, x_test, y_test = get_ml_data(root, subject, sf0, sf1, scouts)

accuracies = []
for rep in range(nreps):
    model = None
    model = create_model()
    model.fit(x_train, y_train, epochs=20, batch_size=128)
    score = model.evaluate(x_test, y_test, batch_size=128)
    accuracies.append(score[1])

Epoch 1/20
216000/216000 [==============================] - 1s 7us/step - loss: 0.6670 - acc: 0.6583
Epoch 2/20
216000/216000 [==============================] - 1s 4us/step - loss: 0.5965 - acc: 0.6785
Epoch 3/20
216000/216000 [==============================] - 1s 4us/step - loss: 0.5956 - acc: 0.6800
Epoch 4/20
216000/216000 [==============================] - 1s 4us/step - loss: 0.5953 - acc: 0.6801
Epoch 5/20
216000/216000 [==============================] - 1s 5us/step - loss: 0.5953 - acc: 0.6801
Epoch 6/20
216000/216000 [==============================] - 1s 5us/step - loss: 0.5952 - acc: 0.6802
Epoch 7/20
216000/216000 [==============================] - 1s 4us/step - loss: 0.5952 - acc: 0.6795
Epoch 8/20
216000/216000 [==============================] - 1s 4us/step - loss: 0.5950 - acc: 0.6803
Epoch 9/20
216000/216000 [==============================] - 1s 4us/step - loss: 0.5951 - acc: 0.6800
Epoch 10/20
216000/216000 [==============================] - 1s 4us/step - loss: 0.5949 - a

216000/216000 [==============================] - 1s 4us/step - loss: 0.5961 - acc: 0.6800
Epoch 20/20
24002/24002 [==============================] - 0s 10us/step
Epoch 1/20
216000/216000 [==============================] - 1s 7us/step - loss: 0.6877 - acc: 0.6598
Epoch 2/20
216000/216000 [==============================] - 1s 4us/step - loss: 0.5957 - acc: 0.6794
Epoch 3/20
216000/216000 [==============================] - 1s 4us/step - loss: 0.5952 - acc: 0.6803
Epoch 4/20
216000/216000 [==============================] - 1s 4us/step - loss: 0.5949 - acc: 0.6804
Epoch 5/20
216000/216000 [==============================] - 1s 4us/step - loss: 0.5948 - acc: 0.6805
Epoch 6/20
216000/216000 [==============================] - 1s 4us/step - loss: 0.5947 - acc: 0.6806
Epoch 7/20
216000/216000 [==============================] - 1s 4us/step - loss: 0.5945 - acc: 0.6802
Epoch 8/20
216000/216000 [==============================] - 1s 4us/step - loss: 0.5945 - acc: 0.6810
Epoch 9/20
216000/216000 [====

<IPython.core.display.Javascript object>

In [20]:
accuracies

[0.6601533205963535,
 0.6596533622992813,
 0.6575285393678588,
 0.6578201816068293,
 0.6598616781487842]

In [24]:
# plot_model?
plot_model(model, to_file='ann_model.png', show_shapes=True, show_layer_names=False)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
plt.plot(y_pred)
plt.show()